torch.autograd是pytorch的自动差分引擎，可以为神经网络训练提供支持

训练神经网络分为两个步骤：

**正向传播**：在正向传播中，NN对正确的输出进行最佳猜测，它通过其每个函数运行输入数据以进行猜测

**反向传播**：在反向传播中，NN根据其猜测中的误差调整其参数，通过从输出向后遍历，收集有关函数参数的误差导数并使用梯度下降来优化参数来实现。



In [4]:
!nvidia-smi

Wed Sep  7 13:51:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

通过模型的每一层运行输入数据进行预测，这是正向传播。

In [6]:
prediction = model(data)

我们使用模型的预测和相应的标签来计算误差。
下一步是通过网络反向传播此误差。当我们在误差变量上调用.backward()时，开始反向传播

In [7]:
loss = (prediction - labels).sum()
loss.backward()

接下来，我们加载一个优化器。
本例中使用SGD,学习率为0.01，动量为0.9

In [9]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)


最后我们调用.step()启动梯度下降，优化器通过.grad中存储的梯度来调整每个参数

In [10]:
optim.step()

假设a和b是神经网络的参数，Q是误差。在NN训练中，我们想要相对于参数的误差。

当我们在Q上调用Q.backward()中显示传递gradient参数，因为它是向量。gradient是与Q形状相同的张量，它表示Q相对于本身的梯度，即![](https://pytorch.apachecn.org/docs/1.7/img/tex4-4.gif)

同样，我们也可以将Q聚合成一个标量，然后隐式地向后调用，例如Q.sum().backward()

In [13]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)
Q = 3*a**3 - b**2

external_grad = torch.tensor([1., 1.])
Q.backward(gradient= external_grad)

梯度现在沉积在a.grad和b.grad中


In [16]:
# check if collected gradients are correct
print(9*a**2 == a.grad)
print(-2*b == b.grad)


tensor([True, True])
tensor([True, True])


torch.autograd跟踪所有将requires_grad标志设置为True的张量的操作，对于不需要梯度的张量，将此属性设置为false，将会从梯度计算DAG中排除。
即使只有一个输入张量具有requires_grad=True，操作的输出张量也将需要梯度。


In [17]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients? : {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")


Does `a` require gradients? : False
Does `b` require gradients?: True


在NN中，不计算梯度的参数通常称为冻结参数，如果实现知道您不需要这些参数的梯度，则冻结模型的一部分很有用（通过减少自动梯度运算，这会带来一些性能优势）
 
从DAG中排除很重要的另一个常见用例是调整预训练网络
在微调中，我们冻结了大部分模型，通常仅修改分类器层以对新标签进行预测，我们来看加载一个预训练的resnet18模型，并冻结所有参数。



In [18]:
from torch import nn, optim

model = torchvision.models.resnet18(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


假设我们要在具有10个标签的新数据集中微调模型。在resnet中，分类器是最后一个线性层model.fc。我们可以简单地将其替换为充当我们分类器的新线性层。

In [19]:
model.fc = nn.Linear(512, 10)


现在除了model.fc的参数外，模型中所有参数都将冻结，计算梯度的唯一参数是model.fc的权重和偏差。

In [20]:
optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)